In [1]:
import sys
sys.path.append('/opt/anaconda3/lib/python3.12/site-packages')
import kagglehub


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("minasameh55/king-country-houses-aa")

print("Path to dataset files:", path)

100%|██████████| 780k/780k [00:00<00:00, 1.55MB/s]

Extracting files...
Path to dataset files: /Users/mustachomacho/.cache/kagglehub/datasets/minasameh55/king-country-houses-aa/versions/1
